# Eigenmode Simulation of a X-Mon Qubit

### Qiskit Metal Design

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["PMIX_MCA_gds"]="hash"

# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from collections import OrderedDict

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from SQDMetal.Comps.Xmon import Xmon
from SQDMetal.Comps.Junctions import JunctionDolanPinStretch

# Set up chip design as planar, multiplanar also available
design = designs.DesignPlanar({}, overwrite_enabled=True)

# Set up chip dimensions 
design.chips.main.size.size_x = '500um'
design.chips.main.size.size_y = '500um'
design.chips.main.size.size_z = '500um'
design.chips.main.size.center_x = '0mm'
design.chips.main.size.center_y = '0mm'

# Create the x-mon
xmon = Xmon(design, 'x-mon', options=Dict(pos_x=0, pos_y=0,
                                    vBar_width='24um', hBar_width='24um', vBar_gap=f'{16}um', hBar_gap=f'{16}um',
                                    cross_width=f'{60*2+24}um', cross_height=f'{60*2+24}um',
                                    gap_up='24um', gap_left='24um', gap_right='24um', gap_down='24um'))

# Create the Josephson junction
JunctionDolanPinStretch(design, 'junction', options=Dict(pin_inputs=Dict(start_pin=Dict(component=f'x-mon',pin='right')),
                                                         dist_extend='24um',
                                                         layer=2,
                                                         finger_width='0.4um', t_pad_size='0.385um',
                                                         squid_width='5.4um', prong_width='0.9um'));

# Rebuild the GUI
gui = MetalGUI(design)
gui.rebuild()

### Palace Eigenmode Simulation 
(make sure to replace the entry to 'palace_dir' with the location of the binary for palace)

In [ ]:
from SQDMetal.PALACE.Eigenmode_Simulation import PALACE_Eigenmode_Simulation
from SQDMetal.Utilities.Materials import MaterialInterface

#Eigenmode Simulation Options
user_defined_options = {
                "mesh_refinement":  0,                             #refines mesh in PALACE - essetially divides every mesh element in half
                "dielectric_material": "silicon",                  #choose dielectric material - 'silicon' or 'sapphire'
                "starting_freq": 5e9,                              #starting frequency in Hz 
                "number_of_freqs": 1,                              #number of eigenmodes to find
                "solns_to_save": 1,                                #number of electromagnetic field visualizations to save
                "solver_order": 2,                                 #increasing solver order increases accuracy of simulation, but significantly increases sim time
                "solver_tol": 1.0e-8,                              #error residual tolerance foriterative solver
                "solver_maxits": 200,                              #number of solver iterations
                "fillet_resolution":12,                             #Number of vertices per quarter turn on a filleted path
                "palace_dir":"~/spack/opt/spack/linux-ubuntu24.04-zen2/gcc-13.3.0/palace-develop-36rxmgzatchgymg5tcbfz3qrmkf4jnmj/bin/palace",#"PATH/TO/PALACE/BINARY",
                "num_cpus": 16
                }

#Create the Palace Eigenmode simulation
eigen_sim = PALACE_Eigenmode_Simulation(name ='x-mon_test',                                         #name of simulation
                                        metal_design = design,                                      #feed in qiskit metal design
                                        sim_parent_directory = "",                                  #choose directory where mesh file, config file and HPC batch file will be saved
                                        mode = 'simPC',                                             #choose simulation mode 'HPC' or 'simPC'                                          
                                        meshing = 'GMSH',                                           #choose meshing 'GMSH' or 'COMSOL'
                                        user_options = user_defined_options,                        #provide options chosen above
                                        create_files = True)                                        #create mesh, config and HPC batch files

#Add in metals from layer 1 of the design file
eigen_sim.add_metallic(1)

#Add in ground plane for simulation
eigen_sim.add_ground_plane()

#Add in the Josephson junction as a lumped port
eigen_sim.create_port_JosephsonJunction('junction', L_J=4.3e-9, C_J=10e-15)

#Fine-mesh x-mon
eigen_sim.fine_mesh_components(['x-mon'], min_size=8e-6, max_size=100e-6, taper_dist_min=10e-6, metals_only=False)

#Sets up the lossy interfaces for MA, SA and MS interfaces 
eigen_sim.setup_EPR_interfaces(metal_air=MaterialInterface('Aluminium-Vacuum'), substrate_air=MaterialInterface('Silicon-Vacuum'), substrate_metal=MaterialInterface('Silicon-Aluminium'))

#Prepares the mesh file and config file
eigen_sim.prepare_simulation()

In [5]:
#inspect the mesh before running the simulation
eigen_sim.open_mesh_gmsh()

In [ ]:
#run the simulation
eigen_sim.run()